# Esercitazione: Utilizzo di Gemini in Google Colab per il Machine Learning con K-means

Questo notebook guida gli studenti nell'utilizzo di Gemini, il modello di AI di Google, all'interno di Google Colab per sviluppare un progetto di Machine Learning con focus sulla clusterizzazione K-means. Seguiremo le principali fasi di un progetto ML:

1. Caricamento e analisi del dataset
2. Pulizia dei dati
3. Feature engineering semplice
4. Realizzazione di un modello di classificazione
5. Implementazione della clusterizzazione K-means

## Come utilizzare questo notebook

- Le caselle di testo contengono i prompt da dare a Gemini
- Le caselle di codice sono dove incollare il codice generato da Gemini
- Segui le istruzioni passo dopo passo per completare l'esercitazione

## Configurazione iniziale

Prima di iniziare, dobbiamo attivare Gemini in Google Colab. Nella barra laterale di Colab, cerca l'icona di Gemini (un rombo colorato) e assicurati che sia attivato.

Installiamo anche le librerie necessarie per il nostro progetto.

In [ ]:
# Installa le librerie necessarie
!pip install pandas numpy matplotlib seaborn scikit-learn

## Fase 1: Caricamento e analisi del dataset

### Prompt per Gemini:

```
Voglio caricare e analizzare il dataset Wine di scikit-learn. Puoi fornirmi il codice per:
1. Caricare il dataset
2. Visualizzare le prime righe
3. Ottenere statistiche descrittive
4. Creare alcuni grafici per esplorare i dati (istogrammi, box plot e matrice di correlazione)
```

Copia il prompt sopra e incollalo nella chat di Gemini. Poi copia il codice generato nella cella seguente.

In [ ]:
# Incolla qui il codice generato da Gemini per caricare e analizzare il dataset Wine

# Esempio di codice che potresti ottenere:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine

# 1. Caricare il dataset
wine = load_wine()
X = wine.data
y = wine.target
feature_names = wine.feature_names
target_names = wine.target_names

# Creiamo un DataFrame per una migliore manipolazione
df = pd.DataFrame(X, columns=feature_names)
df['wine_type'] = [target_names[i] for i in y]

# 2. Visualizzare le prime righe
print("Informazioni sul dataset Wine:")
print(f"Numero di campioni: {X.shape[0]}")
print(f"Numero di feature: {X.shape[1]}")
print(f"Classi: {target_names}")
print("\nPrime 5 righe del dataset:")
print(df.head())

# 3. Ottenere statistiche descrittive
print("\nStatistiche descrittive:")
print(df.describe())

# Distribuzione delle classi
print("\nDistribuzione delle classi:")
print(df['wine_type'].value_counts())

# 4. Creare alcuni grafici per esplorare i dati
# Configurazione del plot
plt.figure(figsize=(15, 10))

# Istogrammi per alcune feature
plt.subplot(2, 2, 1)
sns.histplot(data=df, x='alcohol', hue='wine_type', kde=True)
plt.title('Distribuzione di alcohol per tipo di vino')

plt.subplot(2, 2, 2)
sns.histplot(data=df, x='malic_acid', hue='wine_type', kde=True)
plt.title('Distribuzione di malic_acid per tipo di vino')

plt.subplot(2, 2, 3)
sns.histplot(data=df, x='color_intensity', hue='wine_type', kde=True)
plt.title('Distribuzione di color_intensity per tipo di vino')

plt.subplot(2, 2, 4)
sns.histplot(data=df, x='flavanoids', hue='wine_type', kde=True)
plt.title('Distribuzione di flavanoids per tipo di vino')

plt.tight_layout()
plt.show()

# Box plot per alcune feature
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x='wine_type', y='alcohol', data=df)
plt.title('Box plot di alcohol per tipo di vino')

plt.subplot(2, 2, 2)
sns.boxplot(x='wine_type', y='malic_acid', data=df)
plt.title('Box plot di malic_acid per tipo di vino')

plt.subplot(2, 2, 3)
sns.boxplot(x='wine_type', y='color_intensity', data=df)
plt.title('Box plot di color_intensity per tipo di vino')

plt.subplot(2, 2, 4)
sns.boxplot(x='wine_type', y='flavanoids', data=df)
plt.title('Box plot di flavanoids per tipo di vino')

plt.tight_layout()
plt.show()

# Matrice di correlazione
plt.figure(figsize=(12, 10))
correlation_matrix = df.drop('wine_type', axis=1).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Matrice di correlazione delle feature')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Analisi dei risultati

Osserva i risultati ottenuti. Quali sono le caratteristiche principali del dataset Wine? Quali feature sembrano più utili per distinguere i diversi tipi di vino?

Puoi chiedere a Gemini di aiutarti a interpretare i risultati con il seguente prompt:

```
Puoi aiutarmi a interpretare i risultati dell'analisi del dataset Wine? Quali sono le feature più importanti per distinguere i diversi tipi di vino in base ai grafici e alla matrice di correlazione?
```

## Fase 2: Pulizia dei dati

### Prompt per Gemini:

```
Voglio pulire il dataset Wine. Puoi fornirmi il codice per:
1. Verificare se ci sono valori mancanti
2. Identificare e gestire eventuali outlier
3. Normalizzare le feature (utilizzando StandardScaler)
4. Visualizzare la distribuzione dei dati prima e dopo la normalizzazione
```

Copia il prompt sopra e incollalo nella chat di Gemini. Poi copia il codice generato nella cella seguente.

In [ ]:
# Incolla qui il codice generato da Gemini per la pulizia dei dati

# Esempio di codice che potresti ottenere:
from sklearn.preprocessing import StandardScaler

# 1. Verificare se ci sono valori mancanti
print("Valori mancanti nel dataset:")
print(df.isnull().sum())

# 2. Identificare e gestire eventuali outlier
# Funzione per identificare gli outlier con il metodo IQR
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Verifichiamo gli outlier per ogni feature numerica
numeric_features = df.drop('wine_type', axis=1).columns
outliers_summary = {}

for feature in numeric_features:
    outliers, lower, upper = identify_outliers(df, feature)
    if len(outliers) > 0:
        outliers_summary[feature] = {
            'count': len(outliers),
            'lower_bound': lower,
            'upper_bound': upper
        }
        print(f"Outlier trovati in {feature}: {len(outliers)}")
        print(f"  Limite inferiore: {lower:.2f}, Limite superiore: {upper:.2f}")
    else:
        print(f"Nessun outlier trovato in {feature}")

# Visualizziamo la distribuzione di alcune feature con outlier
features_with_outliers = [feature for feature in outliers_summary.keys() if outliers_summary[feature]['count'] > 0]
if features_with_outliers:
    plt.figure(figsize=(15, 5 * (len(features_with_outliers) + 1) // 2))
    for i, feature in enumerate(features_with_outliers[:4]):  # Limitiamo a 4 feature per brevità
        plt.subplot(2, 2, i+1)
        sns.boxplot(x='wine_type', y=feature, data=df)
        plt.title(f'Box plot di {feature} con outlier')
    plt.tight_layout()
    plt.show()

# Gestione degli outlier: in questo caso, scegliamo di mantenerli poiché il dataset è piccolo
# e gli outlier potrebbero contenere informazioni importanti
print("\nDecisione: Mantenere gli outlier per questo dataset")

# 3. Normalizzare le feature
# Separiamo feature e target
X = df.drop('wine_type', axis=1)
y = df['wine_type']

# Visualizziamo la distribuzione di alcune feature prima della normalizzazione
plt.figure(figsize=(15, 10))
plt.suptitle('Distribuzione delle feature prima della normalizzazione', fontsize=16)

for i, feature in enumerate(X.columns[:4]):  # Visualizziamo solo le prime 4 feature
    plt.subplot(2, 2, i+1)
    sns.histplot(data=X, x=feature, kde=True)
    plt.title(f'Distribuzione di {feature}')

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Inizializziamo lo scaler
scaler = StandardScaler()

# Applichiamo la normalizzazione
X_scaled = scaler.fit_transform(X)

# Convertiamo in DataFrame per una migliore visualizzazione
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 4. Visualizzare la distribuzione dei dati dopo la normalizzazione
plt.figure(figsize=(15, 10))
plt.suptitle('Distribuzione delle feature dopo la normalizzazione', fontsize=16)

for i, feature in enumerate(X_scaled_df.columns[:4]):  # Visualizziamo solo le prime 4 feature
    plt.subplot(2, 2, i+1)
    sns.histplot(data=X_scaled_df, x=feature, kde=True)
    plt.title(f'Distribuzione di {feature} (normalizzata)')

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Verifichiamo la media e la deviazione standard delle feature normalizzate
print("\nStatistiche delle feature normalizzate:")
print(X_scaled_df.describe().loc[['mean', 'std']])

# Aggiungiamo la colonna target al DataFrame normalizzato per usi futuri
X_scaled_df['wine_type'] = y.values
print("\nPrime 5 righe del dataset normalizzato con target:")
print(X_scaled_df.head())

### Riflessione sulla pulizia dei dati

Rifletti sui risultati della pulizia dei dati:
- Ci sono valori mancanti nel dataset?
- Quali feature presentano outlier significativi?
- Come sono cambiate le distribuzioni dopo la normalizzazione?
- Perché la normalizzazione è importante per algoritmi come K-means?

Puoi chiedere a Gemini di spiegarti l'importanza della normalizzazione dei dati per K-means con il seguente prompt:

```
Perché la normalizzazione dei dati è particolarmente importante per l'algoritmo K-means? Cosa potrebbe succedere se applicassimo K-means a dati non normalizzati?
```

## Fase 3: Feature engineering semplice

### Prompt per Gemini:

```
Voglio fare un feature engineering semplice sul dataset Wine. Puoi fornirmi il codice per:
1. Creare feature polinomiali di grado 2 per alcune feature selezionate (ad esempio, alcohol e flavanoids)
2. Creare feature di interazione tra coppie di feature importanti
3. Selezionare le feature più importanti utilizzando la correlazione con la variabile target
4. Visualizzare le nuove feature create
```

Copia il prompt sopra e incollalo nella chat di Gemini. Poi copia il codice generato nella cella seguente.

In [ ]:
# Incolla qui il codice generato da Gemini per il feature engineering

# Esempio di codice che potresti ottenere:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Utilizziamo il dataset originale (non normalizzato) per il feature engineering
X_original = df.drop('wine_type', axis=1)
y_original = df['wine_type']

# Convertiamo le etichette categoriche in numeriche per calcolare la correlazione
le = LabelEncoder()
y_encoded = le.fit_transform(y_original)

# 1. Creare feature polinomiali di grado 2 per alcune feature selezionate
# Selezioniamo alcune feature per cui creare termini quadratici
selected_features = ['alcohol', 'flavanoids', 'color_intensity', 'proline']
X_poly = X_original.copy()

for feature in selected_features:
    X_poly[f"{feature}_squared"] = X_original[feature] ** 2

print("Feature polinomiali create:")
for feature in selected_features:
    print(f"- {feature}_squared")

# 2. Creare feature di interazione tra coppie di feature importanti
# Definiamo alcune coppie di feature per cui creare interazioni
interaction_pairs = [
    ('alcohol', 'malic_acid'),
    ('flavanoids', 'color_intensity'),
    ('alcohol', 'proline'),
    ('od280/od315_of_diluted_wines', 'proline')
]

for feat1, feat2 in interaction_pairs:
    X_poly[f"{feat1}_{feat2}_interaction"] = X_original[feat1] * X_original[feat2]

print("\nFeature di interazione create:")
for feat1, feat2 in interaction_pairs:
    print(f"- {feat1}_{feat2}_interaction")

# 3. Selezionare le feature più importanti utilizzando la correlazione con la variabile target
# Aggiungiamo la variabile target al DataFrame per calcolare la correlazione
X_poly_with_target = X_poly.copy()
X_poly_with_target['target'] = y_encoded

# Calcoliamo la correlazione di ogni feature con la variabile target
correlations = X_poly_with_target.corr()['target'].sort_values(ascending=False)

print("\nCorrelazione delle feature con la variabile target:")
print(correlations.drop('target'))

# Selezioniamo le top 10 feature più correlate con il target
top_features = correlations.drop('target').abs().sort_values(ascending=False).head(10).index.tolist()

print("\nTop 10 feature più correlate con il target:")
for i, feature in enumerate(top_features):
    print(f"{i+1}. {feature} (correlazione: {correlations[feature]:.4f})")

# Creiamo un dataset con solo le feature più importanti
X_selected = X_poly[top_features]

# 4. Visualizzare le nuove feature create
# Visualizziamo la distribuzione delle nuove feature polinomiali
plt.figure(figsize=(15, 10))
plt.suptitle('Distribuzione delle feature polinomiali', fontsize=16)

squared_features = [f for f in X_poly.columns if '_squared' in f]
for i, feature in enumerate(squared_features):
    if i < 4:  # Limitiamo a 4 grafici
        plt.subplot(2, 2, i+1)
        sns.histplot(data=X_poly, x=feature, hue=y_original, kde=True, bins=15)
        plt.title(f'Distribuzione di {feature}')

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Visualizziamo la distribuzione delle nuove feature di interazione
plt.figure(figsize=(15, 10))
plt.suptitle('Distribuzione delle feature di interazione', fontsize=16)

interaction_features = [f for f in X_poly.columns if '_interaction' in f]
for i, feature in enumerate(interaction_features):
    if i < 4:  # Limitiamo a 4 grafici
        plt.subplot(2, 2, i+1)
        sns.histplot(data=X_poly, x=feature, hue=y_original, kde=True, bins=15)
        plt.title(f'Distribuzione di {feature}')

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Visualizziamo la matrice di correlazione delle feature selezionate
plt.figure(figsize=(12, 10))
correlation_matrix = X_selected.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Matrice di correlazione delle feature selezionate')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Prepariamo il dataset finale per i modelli
# Normalizziamo le feature selezionate
scaler = StandardScaler()
X_selected_scaled = scaler.fit_transform(X_selected)

print("\nDimensioni del dataset finale:")
print(f"Numero di campioni: {X_selected_scaled.shape[0]}")
print(f"Numero di feature: {X_selected_scaled.shape[1]}")

### Riflessione sul feature engineering

Rifletti sui risultati del feature engineering:
- Quali nuove feature sembrano più utili per distinguere i tipi di vino?
- Come sono cambiate le correlazioni dopo la creazione delle nuove feature?
- Quali feature originali e derivate sono state selezionate come più importanti?

Puoi chiedere a Gemini di spiegarti l'importanza del feature engineering nel machine learning con il seguente prompt:

```
Perché il feature engineering è importante nel machine learning? Quali sono i vantaggi di creare feature polinomiali e di interazione? In quali casi potrebbe essere controproducente?
```

## Fase 4: Realizzazione di un modello di classificazione

### Prompt per Gemini:

```
Voglio creare un modello di classificazione per il dataset Wine utilizzando le feature selezionate. Puoi fornirmi il codice per:
1. Dividere il dataset in training e test set (80% training, 20% test)
2. Addestrare un modello di Random Forest
3. Valutare le performance del modello (accuracy, matrice di confusione, report di classificazione)
4. Visualizzare l'importanza delle feature nel modello
```

Copia il prompt sopra e incollalo nella chat di Gemini. Poi copia il codice generato nella cella seguente.

In [ ]:
# Incolla qui il codice generato da Gemini per la creazione del modello

# Esempio di codice che potresti ottenere:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Utilizziamo le feature selezionate e normalizzate
X = X_selected_scaled
y = y_original.values  # Utilizziamo le etichette originali

# 1. Dividere il dataset in training e test set (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verifichiamo le dimensioni dei set
print(f"Dimensioni X_train: {X_train.shape}")
print(f"Dimensioni X_test: {X_test.shape}")
print(f"Dimensioni y_train: {len(y_train)}")
print(f"Dimensioni y_test: {len(y_test)}")

# Verifichiamo la distribuzione delle classi nei set di training e test
print("\nDistribuzione delle classi nel set di training:")
train_class_counts = pd.Series(y_train).value_counts()
for class_name, count in zip(train_class_counts.index, train_class_counts.values):
    print(f"- {class_name}: {count}")

print("\nDistribuzione delle classi nel set di test:")
test_class_counts = pd.Series(y_test).value_counts()
for class_name, count in zip(test_class_counts.index, test_class_counts.values):
    print(f"- {class_name}: {count}")

# 2. Addestrare un modello di Random Forest
# Inizializziamo il modello
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Addestriamo il modello
rf_model.fit(X_train, y_train)

# Facciamo previsioni sul set di test
y_pred = rf_model.predict(X_test)

# 3. Valutare le performance del modello
# Calcoliamo l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy del modello: {accuracy:.4f}")

# Calcoliamo la matrice di confusione
cm = confusion_matrix(y_test, y_pred)

# Visualizziamo la matrice di confusione
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=wine.target_names, yticklabels=wine.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione')
plt.show()

# Generiamo il report di classificazione
report = classification_report(y_test, y_pred, target_names=wine.target_names)
print("\nReport di classificazione:")
print(report)

# 4. Visualizzare l'importanza delle feature nel modello
# Otteniamo l'importanza delle feature
feature_importances = rf_model.feature_importances_

# Creiamo un DataFrame per visualizzare meglio
feature_importance_df = pd.DataFrame({
    'Feature': top_features,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

print("\nImportanza delle feature nel modello Random Forest:")
print(feature_importance_df)

# Visualizziamo l'importanza delle feature
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Importanza delle Feature nel Random Forest')
plt.tight_layout()
plt.show()

### Riflessione sul modello

Rifletti sui risultati del modello:
- Qual è l'accuracy del modello? È un buon risultato?
- Ci sono classi che il modello fatica a distinguere?
- Quali sono le feature più importanti per la classificazione?
- Come si confrontano le feature originali con quelle create durante il feature engineering?

Puoi chiedere a Gemini di spiegarti come funziona l'algoritmo Random Forest e come interpretare i risultati con il seguente prompt:

```
Puoi spiegarmi come funziona l'algoritmo Random Forest? Come dovrei interpretare la matrice di confusione e l'importanza delle feature? Quali sono i vantaggi di Random Forest rispetto ad altri algoritmi di classificazione?
```

## Fase 5: Implementazione della clusterizzazione K-means

### Prompt per Gemini:

```
Voglio implementare l'algoritmo K-means sul dataset Wine. Puoi fornirmi il codice per:
1. Applicare l'algoritmo K-means con k=3 (poiché sappiamo che ci sono 3 tipi di vino)
2. Visualizzare i cluster utilizzando PCA per ridurre a 2 dimensioni
3. Confrontare i cluster ottenuti con le etichette reali
4. Determinare il numero ottimale di cluster usando il metodo del gomito e il silhouette score
5. Analizzare le caratteristiche di ciascun cluster
```

Copia il prompt sopra e incollalo nella chat di Gemini. Poi copia il codice generato nella cella seguente.

In [ ]:
# Incolla qui il codice generato da Gemini per la clusterizzazione K-means

# Esempio di codice che potresti ottenere:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, silhouette_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Utilizziamo le feature selezionate e normalizzate
X = X_selected_scaled
y = y_original.values

# Convertiamo le etichette categoriche in numeriche per il confronto
le = LabelEncoder()
y_numeric = le.fit_transform(y)

# 1. Applicare l'algoritmo K-means con k=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X)

# 2. Visualizzare i cluster utilizzando PCA per ridurre a 2 dimensioni
# Applichiamo PCA per ridurre a 2 dimensioni
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Creiamo un DataFrame con i risultati
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
df_pca['Cluster'] = cluster_labels
df_pca['True_Label'] = y

# Visualizziamo i cluster
plt.figure(figsize=(15, 6))

# Plot dei cluster predetti
plt.subplot(1, 2, 1)
sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=df_pca, palette='viridis', s=100, alpha=0.7)

# Aggiungiamo i centroidi
centroids_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=300, c='red', marker='X', label='Centroids')
plt.title('Cluster K-means (k=3)')
plt.legend()

# Plot delle etichette reali
plt.subplot(1, 2, 2)
sns.scatterplot(x='PC1', y='PC2', hue='True_Label', data=df_pca, palette='Set1', s=100, alpha=0.7)
plt.title('Etichette reali')
plt.legend()

plt.tight_layout()
plt.show()

# 3. Confrontare i cluster ottenuti con le etichette reali
# Calcoliamo l'Adjusted Rand Index (ARI)
ari = adjusted_rand_score(y_numeric, cluster_labels)
print(f"Adjusted Rand Index: {ari:.4f}")
# L'ARI varia da -1 a 1, dove 1 indica un clustering perfetto

# Calcoliamo il Silhouette Score
silhouette = silhouette_score(X, cluster_labels)
print(f"Silhouette Score: {silhouette:.4f}")
# Il Silhouette Score varia da -1 a 1, dove valori più alti indicano cluster meglio definiti

# Creiamo una tabella di contingenza per vedere come i cluster si mappano alle etichette reali
contingency_table = pd.crosstab(y, pd.Series(cluster_labels, name='Cluster'))
print("\nTabella di contingenza (etichette reali vs cluster):")
print(contingency_table)

# Visualizziamo la tabella di contingenza come heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='Blues')
plt.title('Tabella di contingenza: Etichette reali vs Cluster')
plt.show()

# 4. Determinare il numero ottimale di cluster usando il metodo del gomito e il silhouette score
# Calcoliamo l'inerzia (somma dei quadrati delle distanze) per diversi valori di k
inertia = []
silhouette_scores = []
ari_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, cluster_labels))
    ari_scores.append(adjusted_rand_score(y_numeric, cluster_labels))

# Visualizziamo il grafico dell'inerzia (metodo del gomito)
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(k_range, inertia, 'o-', markersize=8)
plt.xlabel('Numero di cluster (k)')
plt.ylabel('Inerzia')
plt.title('Metodo del gomito')
plt.grid(True, linestyle='--', alpha=0.7)

# Visualizziamo il grafico del silhouette score
plt.subplot(1, 3, 2)
plt.plot(k_range, silhouette_scores, 'o-', markersize=8)
plt.xlabel('Numero di cluster (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score')
plt.grid(True, linestyle='--', alpha=0.7)

# Visualizziamo il grafico dell'ARI
plt.subplot(1, 3, 3)
plt.plot(k_range, ari_scores, 'o-', markersize=8)
plt.xlabel('Numero di cluster (k)')
plt.ylabel('Adjusted Rand Index')
plt.title('Adjusted Rand Index')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Troviamo il k ottimale in base al silhouette score
optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
print(f"Numero ottimale di cluster in base al silhouette score: {optimal_k_silhouette}")

# Troviamo il k ottimale in base all'ARI
optimal_k_ari = k_range[np.argmax(ari_scores)]
print(f"Numero ottimale di cluster in base all'ARI: {optimal_k_ari}")

# 5. Analizzare le caratteristiche di ciascun cluster
# Aggiungiamo le etichette dei cluster al dataset originale
df_with_clusters = df.copy()
df_with_clusters['cluster'] = cluster_labels

# Calcoliamo le medie delle feature per ciascun cluster
cluster_means = df_with_clusters.groupby('cluster').mean()
print("\nMedia delle feature per ciascun cluster:")
print(cluster_means)

# Visualizziamo le caratteristiche di ciascun cluster per alcune feature selezionate
selected_features_for_viz = ['alcohol', 'malic_acid', 'flavanoids', 'color_intensity', 'proline']

# Creiamo un radar chart per visualizzare le caratteristiche dei cluster
# Normalizziamo i valori per il radar chart
cluster_means_normalized = cluster_means.copy()
for feature in cluster_means.columns:
    min_val = cluster_means[feature].min()
    max_val = cluster_means[feature].max()
    if max_val > min_val:  # Evita divisione per zero
        cluster_means_normalized[feature] = (cluster_means[feature] - min_val) / (max_val - min_val)

# Selezioniamo solo le feature che vogliamo visualizzare
cluster_means_normalized = cluster_means_normalized[selected_features_for_viz]

# Creiamo il radar chart
categories = selected_features_for_viz
N = len(categories)

# Creiamo gli angoli per il radar chart
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Chiudiamo il cerchio

# Inizializziamo il plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

# Aggiungiamo le etichette
plt.xticks(angles[:-1], categories, size=12)

# Disegniamo i limiti del grafico
ax.set_rlabel_position(0)
plt.yticks([0.25, 0.5, 0.75], ["0.25", "0.5", "0.75"], color="grey", size=10)
plt.ylim(0, 1)

# Plottiamo ciascun cluster
for cluster_id in range(3):
    values = cluster_means_normalized.loc[cluster_id].values.tolist()
    values += values[:1]  # Chiudiamo il cerchio
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {cluster_id}')
    ax.fill(angles, values, alpha=0.1)

# Aggiungiamo la legenda
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.title('Caratteristiche dei cluster', size=15)
plt.show()

# Visualizziamo anche i box plot per alcune feature selezionate
plt.figure(figsize=(15, 10))
for i, feature in enumerate(selected_features_for_viz):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='cluster', y=feature, data=df_with_clusters)
    plt.title(f'Distribuzione di {feature} per cluster')

plt.tight_layout()
plt.show()

# Confrontiamo la distribuzione dei tipi di vino in ciascun cluster
plt.figure(figsize=(12, 6))
wine_type_by_cluster = pd.crosstab(df_with_clusters['cluster'], df_with_clusters['wine_type'], normalize='index') * 100
wine_type_by_cluster.plot(kind='bar', stacked=True)
plt.title('Distribuzione dei tipi di vino in ciascun cluster (%)')
plt.xlabel('Cluster')
plt.ylabel('Percentuale')
plt.legend(title='Tipo di vino')
plt.show()

print("\nDistribuzione dei tipi di vino in ciascun cluster (%):\n")
print(wine_type_by_cluster)

### Riflessione sulla clusterizzazione K-means

Rifletti sui risultati della clusterizzazione K-means:
- I cluster identificati da K-means corrispondono ai tipi di vino reali?
- Qual è il numero ottimale di cluster secondo il metodo del gomito e il silhouette score?
- Quali sono le caratteristiche distintive di ciascun cluster?
- Come si confronta la clusterizzazione non supervisionata con la classificazione supervisionata?

Puoi chiedere a Gemini di spiegarti come funziona l'algoritmo K-means e come interpretare i risultati con il seguente prompt:

```
Puoi spiegarmi come funziona l'algoritmo K-means? Come dovrei interpretare l'Adjusted Rand Index, il Silhouette Score e il metodo del gomito? Quali sono i limiti di K-means e in quali situazioni potrebbe non funzionare bene?
```

## Conclusioni e sfide aggiuntive

Congratulazioni! Hai completato l'esercitazione sull'utilizzo di Gemini in Google Colab per un progetto di Machine Learning con focus sulla clusterizzazione K-means. Hai seguito le principali fasi di un progetto ML:

1. Caricamento e analisi del dataset
2. Pulizia dei dati
3. Feature engineering semplice
4. Realizzazione di un modello di classificazione
5. Implementazione della clusterizzazione K-means

### Confronto tra classificazione supervisionata e clusterizzazione non supervisionata

In questo notebook, hai avuto l'opportunità di confrontare due approcci fondamentali del machine learning:
- **Classificazione supervisionata** (Random Forest): utilizza le etichette per addestrare un modello che può predire la classe di nuovi dati
- **Clusterizzazione non supervisionata** (K-means): raggruppa i dati in base alla loro similarità senza utilizzare le etichette

Hai potuto osservare come K-means sia in grado di identificare pattern nei dati senza conoscere le etichette reali, e come questi pattern possano corrispondere (o meno) alle classi reali.

### Sfide aggiuntive

Se vuoi approfondire ulteriormente, ecco alcune sfide che puoi provare:

1. Prova a utilizzare un dataset diverso (ad esempio, Breast Cancer o Digits da scikit-learn)
2. Implementa altri algoritmi di clustering (DBSCAN, Hierarchical Clustering, Gaussian Mixture Models)
3. Esplora tecniche di riduzione della dimensionalità diverse da PCA (t-SNE, UMAP)
4. Implementa una pipeline completa di ML con GridSearchCV per l'ottimizzazione degli iperparametri di K-means
5. Crea una visualizzazione interattiva dei risultati con Plotly

Per ciascuna di queste sfide, puoi chiedere aiuto a Gemini con prompt specifici.

### Prompt per sfide aggiuntive

Ecco alcuni prompt che puoi utilizzare per le sfide aggiuntive:

```
Puoi fornirmi il codice per implementare DBSCAN sul dataset Wine e confrontare i risultati con K-means?
```

```
Puoi mostrarmi come implementare t-SNE per visualizzare il dataset Wine in 2D e confrontarlo con PCA?
```

```
Puoi fornirmi il codice per implementare una pipeline di ML con GridSearchCV per ottimizzare gli iperparametri di K-means sul dataset Wine?
```

```
Puoi mostrarmi come creare una visualizzazione interattiva dei risultati di clustering con Plotly?
```

```
Puoi spiegarmi le differenze tra K-means, DBSCAN e Hierarchical Clustering e quando è meglio utilizzare ciascun algoritmo?
```